In [1]:
!git clone https://github.com/crockwell/Cap3D.git
%cd Cap3D/text-to-3D/shap-e


Cloning into 'Cap3D'...
remote: Enumerating objects: 818, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 818 (delta 165), reused 159 (delta 159), pack-reused 641 (from 1)
Receiving objects: 100% (818/818), 106.44 MiB | 10.20 MiB/s, done.
Resolving deltas: 100% (464/464), done.
Updating files: 100% (157/157), done.
/content/Cap3D/text-to-3D/shap-e


In [2]:
!python --version

Python 3.11.12


In [ ]:
!pip install -e .
!pip install gradio


Obtaining file:///content/Cap3D/text-to-3D/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-tqqgrtt2/clip_a990866bdd954d46a22789512687be48
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-tqqgrtt2/clip_a990866bdd954d46a22789512687be48
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/

In [ ]:
# Install gdown if not already installed
!pip install -q gdown

# Download the file using the file ID
!gdown --id 1GQOZFfAMJ4parzt6Amjvy-5Wzqb5CpxC --output myfile.zip

# Unzip the downloaded file
!unzip myfile.zip -d myfolder


In [ ]:

!cp -r myfolder/fineTune/model_ckpts .
!rm -rf myfolder
!rm myfile.zip



In [ ]:
import torch
from shap_e.models.download import load_model
import os

os.makedirs("fine_tuned_models",exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load full models (only once)
text_model = load_model("text300M", device=device)
transmitter = load_model("transmitter", device=device)

# Save their state_dicts
torch.save(text_model.state_dict(), "./fine_tuned_models/text300M_state.pth")
torch.save(transmitter.state_dict(), "./fine_tuned_models/transmitter_state.pth")


**No Need to Run this Cell**

In [ ]:
# ==============================================================================
# Copyright (c) 2023 Tiange Luo, tiange.cs@gmail.com
# Last modified: September 20, 2023
#
# This code is licensed under the MIT License.
# ==============================================================================
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import decode_latent_mesh
import os
import time
import random
from IPython import embed
import pickle
import pandas as pd

# ==== Set configuration variables directly ====
ckpt = 'shapE_finetuned_with_825kdata.pth'
save_name = 'Cap3D_test1_meshes'
test_type = '2k'  # can be '300' or '2k'
# =============================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

xm = load_model("transmitter", device=device)
xm.load_state_dict(torch.load("./fine_tuned_models/transmitter_state.pth", map_location=device))
model = load_model('text300M', device=device)

# Load fine-tuned weights
model.load_state_dict(torch.load(f'./model_ckpts/{ckpt}', map_location=device)['model_state_dict'])

diffusion = diffusion_from_config(load_config('diffusion'))

batch_size = 1
guidance_scale = 20.0

test_uids = pickle.load(open(f'../example_material/test_uids_{test_type}.pkl', 'rb'))
captions = pd.read_csv('../example_material/Cap3D_automated_Objaverse.csv', header=None)

outdir = f'./shapE_inference/{save_name}'
os.makedirs(outdir, exist_ok=True)

print('start generation')

prompt = "A futuristic car"
latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

with torch.no_grad():
    size = 512
    gen_mesh = decode_latent_mesh(xm, latents).tri_mesh()
    save_path = os.path.join(outdir, f'{save_name}.ply')
    with open(save_path, 'wb') as f:
        gen_mesh.write_ply(f)

print(f"Generation complete. Mesh saved to {save_path}")


start generation


  0%|          | 0/64 [00:00<?, ?it/s]

/content/Cap3D/text-to-3D/shap-e/shap_e/models/stf/renderer.py:279: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/Cap3D/text-to-3D/shap-e/shap_e/models/stf/renderer.py:280: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


Generation complete. Mesh saved to ./shapE_inference/Cap3D_test1_meshes/Cap3D_test1_meshes.ply


**DEFINITELY RUN THIS CELL**

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.0 MB/s eta 0:00:00


Gradio Website v1.0

In [ ]:
# -------------------------
# 🧠 Imports and Setup
# -------------------------
import torch
import os
import gradio as gr
import tempfile

from shap_e.models.download import load_model, load_config
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.diffusion.sample import sample_latents
from shap_e.util.notebooks import (
    decode_latent_mesh,
    create_pan_cameras,
    decode_latent_images,
)

# -------------------------
# 🖥️ Device Setup
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------
# 🔧 Load Fine-Tuned Models
# -------------------------

print("✅ Loading models...")
text_model = load_model('text300M', device=device)
ckpt = "shapE_finetuned_with_825kdata.pth"
# Load fine-tuned weights
text_model.load_state_dict(torch.load(f'./model_ckpts/{ckpt}', map_location=device)['model_state_dict'])

transmitter = load_model("transmitter", device=device)
transmitter.load_state_dict(torch.load("./fine_tuned_models/transmitter_state.pth", map_location=device))
diffusion = diffusion_from_config(load_config("diffusion"))

# -------------------------
# ✨ Mesh + GIF Generation Function
# -------------------------
def generate_mesh_and_gifs(prompt, batch_size):
    output_dir = tempfile.mkdtemp()
    ply_paths = []
    gif_paths = []

    latents = sample_latents(
        batch_size=batch_size,
        model=text_model,
        diffusion=diffusion,
        guidance_scale=20.0,
        model_kwargs=dict(texts=[prompt] * batch_size),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=64,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )

    for i, latent in enumerate(latents):
        # Save .ply mesh
        ply_path = os.path.join(output_dir, f"{prompt.replace(' ', '_')}_{i}.ply")
        with open(ply_path, "wb") as f:
            decode_latent_mesh(transmitter, latent).tri_mesh().write_ply(f)
        ply_paths.append(ply_path)

        # Save high-res GIF
        cameras = create_pan_cameras(size=256, device=device)
        images = decode_latent_images(transmitter, latent, cameras, rendering_mode="nerf")

        gif_path = os.path.join(output_dir, f"{prompt.replace(' ', '_')}_{i}.gif")
        images[0].save(gif_path, save_all=True, append_images=images[1:], duration=100, loop=0)
        gif_paths.append(gif_path)

    return gif_paths, ply_paths

# -------------------------
# 🌐 Gradio UI
# -------------------------
iface = gr.Interface(
    fn=generate_mesh_and_gifs,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="e.g. a realistic fantasy sword with intricate design"),
        gr.Slider(minimum=1, maximum=4, step=1, value=1, label="Batch Size (Number of Meshes)")
    ],
    outputs=[
        gr.Gallery(label="🌀 High-Res Rotating GIFs", columns=2, height=320),
        gr.File(label="📦 Download .PLY Mesh Files", file_types=[".ply"], file_count="multiple"),
    ],
    title="Shap-E Fine Tuned 3D Mesh Generator (GIF + Mesh Download)",
    description="Generate beautiful high-res 3D mesh renders as GIFs and download the .ply files using your fine-tuned Shap-E model. This model has been fine tuned Cap3D style using Objaverse dataset.",
)

iface.launch(debug=True)


✅ Loading models...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bdef1081c5c432d122.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://bdef1081c5c432d122.gradio.live


In [ ]:
# -------------------------
# 🧠 Imports and Setup
# -------------------------
import torch
import os
import gradio as gr
import tempfile

from shap_e.models.download import load_model, load_config
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.diffusion.sample import sample_latents
from shap_e.util.notebooks import (
    decode_latent_mesh,
    create_pan_cameras,
    decode_latent_images,
)

# -------------------------
# 🖥️ Device Setup
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------
# 🔧 Load Base + Fine-Tuned Models
# -------------------------

print("✅ Loading models...")

# Load base model
base_text_model = load_model('text300M', device=device)
base_text_model.load_state_dict(torch.load('./fine_tuned_models/text300M_state.pth', map_location=device))

# Load fine-tuned model
fine_tuned_text_model = load_model('text300M', device=device)
fine_tuned_text_model.load_state_dict(torch.load('./model_ckpts/shapE_finetuned_with_825kdata.pth', map_location=device)['model_state_dict'])

# Load transmitter + diffusion
transmitter = load_model("transmitter", device=device)
transmitter.load_state_dict(torch.load("./fine_tuned_models/transmitter_state.pth", map_location=device))
diffusion = diffusion_from_config(load_config("diffusion"))

# -------------------------
# ✨ Mesh + GIF Generation Function
# -------------------------
def generate_mesh_and_gifs(prompt, batch_size):
    output_dir = tempfile.mkdtemp()

    gif_paths_base = []
    gif_paths_finetuned = []

    ply_paths_base = []
    ply_paths_finetuned = []

    def generate_for_model(model, tag):
        latents = sample_latents(
            batch_size=batch_size,
            model=model,
            diffusion=diffusion,
            guidance_scale=20.0,
            model_kwargs=dict(texts=[prompt] * batch_size),
            progress=True,
            clip_denoised=True,
            use_fp16=True,
            use_karras=True,
            karras_steps=64,
            sigma_min=1e-3,
            sigma_max=160,
            s_churn=0,
        )

        gifs = []
        plys = []

        for i, latent in enumerate(latents):
            name_prefix = f"{prompt.replace(' ', '_')}_{tag}_{i}"
            # Save .ply
            ply_path = os.path.join(output_dir, f"{name_prefix}.ply")
            with open(ply_path, "wb") as f:
                decode_latent_mesh(transmitter, latent).tri_mesh().write_ply(f)
            plys.append(ply_path)

            # Save GIF
            cameras = create_pan_cameras(size=256, device=device)
            images = decode_latent_images(transmitter, latent, cameras, rendering_mode="nerf")
            gif_path = os.path.join(output_dir, f"{name_prefix}.gif")
            images[0].save(gif_path, save_all=True, append_images=images[1:], duration=100, loop=0)
            gifs.append(gif_path)

        return gifs, plys

    gif_paths_base, ply_paths_base = generate_for_model(base_text_model, "base")
    gif_paths_finetuned, ply_paths_finetuned = generate_for_model(fine_tuned_text_model, "fine_tuned")

    # Return both sets
    return (
        gif_paths_base + gif_paths_finetuned,
        ply_paths_base + ply_paths_finetuned
    )

# -------------------------
# 🌐 Gradio UI
# -------------------------
iface = gr.Interface(
    fn=generate_mesh_and_gifs,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="e.g. a realistic fantasy sword with intricate design"),
        gr.Slider(minimum=1, maximum=2, step=1, value=1, label="Batch Size (Number of Meshes per Model)")
    ],
    outputs=[
        gr.Gallery(label="🌀 Base vs Fine-Tuned: High-Res GIFs", columns=4, height=320),
        gr.File(label="📦 Download All .PLY Meshes", file_types=[".ply"], file_count="multiple"),
    ],
    title="Shap-E Fine Tuned 3D Mesh Generator (GIF + Mesh Download)",
    description="Generate beautiful high-res 3D mesh renders as GIFs and download the .ply files using your fine-tuned Shap-E model. This model has been fine tuned Cap3D style using Objaverse dataset.",
    )

iface.launch(debug=True)


**VERSION 3.0**

In [ ]:
# -------------------------
# 🧠 Imports and Setup
# -------------------------
import torch
import os
import gradio as gr
import tempfile
from PIL import Image

from shap_e.models.download import load_model, load_config
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.diffusion.sample import sample_latents
from shap_e.util.notebooks import (
    decode_latent_mesh,
    create_pan_cameras,
    decode_latent_images,
)

# -------------------------
# 🖥️ Device Setup
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------
# 🔧 Load Models
# -------------------------
print("✅ Loading models...")

base_text_model = load_model('text300M', device=device)
base_text_model.load_state_dict(torch.load('./fine_tuned_models/text300M_state.pth', map_location=device))

fine_tuned_text_model = load_model('text300M', device=device)
fine_tuned_text_model.load_state_dict(torch.load('./model_ckpts/shapE_finetuned_with_825kdata.pth', map_location=device)['model_state_dict'])

transmitter = load_model("transmitter", device=device)
transmitter.load_state_dict(torch.load("./fine_tuned_models/transmitter_state.pth", map_location=device))

diffusion = diffusion_from_config(load_config("diffusion"))

# -------------------------
# ✨ Mesh + GIF Generator
# -------------------------
def generate_mesh_and_gifs(prompt, batch_size, karras_steps, guidance_scale, sigma_min, sigma_max):
    output_dir = tempfile.mkdtemp()

    def generate_for_model(model, tag):
        latents = sample_latents(
            batch_size=batch_size,
            model=model,
            diffusion=diffusion,
            guidance_scale=guidance_scale,
            model_kwargs=dict(texts=[prompt] * batch_size),
            progress=True,
            clip_denoised=True,
            use_fp16=True,
            use_karras=True,
            karras_steps=karras_steps,
            sigma_min=sigma_min,
            sigma_max=sigma_max,
            s_churn=0,
        )

        gifs = []
        plys = []

        for i, latent in enumerate(latents):
            name_prefix = f"{prompt.replace(' ', '_')}_{tag}_{i}"

            ply_path = os.path.join(output_dir, f"{name_prefix}.ply")
            with open(ply_path, "wb") as f:
                decode_latent_mesh(transmitter, latent).tri_mesh().write_ply(f)
            plys.append(ply_path)

            cameras = create_pan_cameras(size=256, device=device)
            images = decode_latent_images(transmitter, latent, cameras, rendering_mode="nerf")

            gif_path = os.path.join(output_dir, f"{name_prefix}.gif")

            if tag == "base":
                images = [img.convert("P", palette=Image.ADAPTIVE, colors=64) for img in images]
                images[0].save(
                    gif_path,
                    save_all=True,
                    append_images=images[1:],
                    duration=200,
                    loop=0,
                    optimize=True,
                    disposal=2
                )
            else:
                images[0].save(
                    gif_path,
                    save_all=True,
                    append_images=images[1:],
                    duration=200,
                    loop=0,
                    optimize=False
                )

            gifs.append(gif_path)

        return gifs, plys

    gif_paths_base, ply_paths_base = generate_for_model(base_text_model, "base")
    gif_paths_finetuned, ply_paths_finetuned = generate_for_model(fine_tuned_text_model, "fine_tuned")

    return (
        [gr.update(value=gif_paths_base, label="Base Model GIFs"), gr.update(value=gif_paths_finetuned, label="Fine-Tuned Model GIFs")],
        ply_paths_base + ply_paths_finetuned
    )

# -------------------------
# 🌐 Gradio UI
# -------------------------
with gr.Blocks(title="Shap-E Fine Tuned Viewer") as iface:
    gr.Markdown("""
    # 🔬 Shap-E Fine Tuned 3D Mesh Generator
    Generate high-res 3D meshes from prompts. Compare base vs fine-tuned Shap-E model.
    """)

    with gr.Row():
        prompt = gr.Textbox(label="Prompt", placeholder="e.g. a realistic fantasy sword")
        batch_size = gr.Slider(minimum=1, maximum=2, value=1, step=1, label="Batch Size")

    with gr.Row():
        karras_steps = gr.Slider(minimum=32, maximum=128, value=64, step=8, label="Karras Steps")
        guidance_scale = gr.Slider(minimum=1.0, maximum=30.0, value=20.0, step=1.0, label="Guidance Scale")

    with gr.Row():
        sigma_min = gr.Slider(minimum=1e-4, maximum=1e-2, value=1e-3, step=1e-4, label="Sigma Min")
        sigma_max = gr.Slider(minimum=10, maximum=200, value=160, step=10, label="Sigma Max")

    generate_button = gr.Button("Generate 3D Meshes + GIFs")
    error_box = gr.Textbox(label="Errors", visible=False)

    with gr.Row():
        with gr.Column():
            gallery_base = gr.Gallery(label="Base Model GIFs", show_label=True, columns=2, height=320)
        with gr.Column():
            gallery_finetuned = gr.Gallery(label="Fine-Tuned Model GIFs", show_label=True, columns=2, height=320)

    file_output = gr.Files(label="Download All Meshes")

    def wrapper(prompt, batch_size, karras_steps, guidance_scale, sigma_min, sigma_max):
        try:
            (galleries, ply_paths) = generate_mesh_and_gifs(
                prompt, batch_size, int(karras_steps), float(guidance_scale), float(sigma_min), float(sigma_max)
            )
            return galleries[0], galleries[1], ply_paths, gr.update(visible=False)
        except Exception as e:
            return [], [], [], gr.update(value=str(e), visible=True)

    generate_button.click(
        fn=wrapper,
        inputs=[prompt, batch_size, karras_steps, guidance_scale, sigma_min, sigma_max],
        outputs=[gallery_base, gallery_finetuned, file_output, error_box],
    )

iface.launch(debug=True)
